<a href="https://colab.research.google.com/github/shashank-m/covid_mining_papers/blob/master/word_doc2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import codecs
from tqdm import tqdm
import spacy
import matplotlib.pyplot as plt
import nltk
from nltk.util import ngrams

In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
torch.manual_seed(1)

In [4]:

is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


In [5]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
test_sentence = """<UNK> <UNK> When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold. <UNK> <UNK>"""

In [0]:

train_data=[]
# train_labels=[]

In [0]:
#CBOW model.
import re
all_tokens=re.findall(r'\S+',test_sentence)
penta_gram=ngrams(all_tokens,5)
for window in penta_gram:
  window=list(window)
  test_word=window[2]
  train_words=window[:2]+window[3:]
  train_data.append((train_words,test_word))

In [0]:

vocab=set(all_tokens)
vocab_size=len(set(all_tokens))
word_to_idx={}
for i,word in enumerate(vocab):
  word_to_idx[word]=i

In [0]:
context_data=[]
target_data=[]
for context,target in train_data:
  inputs=torch.LongTensor([word_to_idx[word] for word in context]).view(1,-1) 
  target_vector=torch.LongTensor([word_to_idx[target]])
  context_data.append(inputs)
  target_data.append(target_vector)
# train_data  

In [0]:
X_train=torch.cat(context_data,0) # put all training_data into a single tensor so that it can be batched by Dataloader.

In [0]:
X_train_label=torch.cat(target_data,0)

In [0]:
batch_size=1
training=TensorDataset(X_train,X_train_label)
train_loader=DataLoader(training,batch_size=batch_size,drop_last=True)

In [89]:
class cbow(nn.Module):
  def __init__(self,vocab_size,embed_dim,context,hidden_dim,batch_size):
    super(cbow,self).__init__()
    self.bs=batch_size
    self.c=context
    self.embeddings=nn.Embedding(vocab_size,embed_dim)
    self.linear1=nn.Linear(embed_dim,hidden_dim)
    self.linear2=nn.Linear(hidden_dim,vocab_size)
    
  def forward(self,inputs):
    # if inputs.shape[0]==self.bs:
      embeds=self.embeddings(inputs)
      a_1=F.relu(self.linear1(embeds).sum(axis=1)/self.c)
      out=self.linear2(a_1)
      return out

embed_dim=300
model=cbow(vocab_size,embed_dim,4,50,batch_size)
criterion=nn.CrossEntropyLoss()
lr=1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

model.state_dict

<bound method Module.state_dict of cbow(
  (embeddings): Embedding(98, 300)
  (linear1): Linear(in_features=300, out_features=50, bias=True)
  (linear2): Linear(in_features=50, out_features=98, bias=True)
)>

In [0]:
model.embeddings(torch.LongTensor([2])) # embedding of the word winters before training.

**Note that here training data is generated from only one sentence.We run the cbow model over 5 epochs and it looks like the model is learning something as the loss decreases every epoch. If we can overfit on this small data that means out training process works. We just need to collect more training data**

In [90]:
epochs=20
for j in range(epochs):
  for i,(x,y) in enumerate(train_loader):
    model.zero_grad()
    out=model(x)

    loss=criterion(out,y)
    loss.backward()

    optimizer.step()
  print('After epoch {},loss={}'.format(j+1,loss))    

After epoch 1,loss=4.686923027038574
After epoch 2,loss=3.713883876800537
After epoch 3,loss=2.426980495452881
After epoch 4,loss=0.8109285235404968
After epoch 5,loss=0.17440193891525269
After epoch 6,loss=0.10496030002832413
After epoch 7,loss=0.06620841473340988
After epoch 8,loss=0.047774963080883026
After epoch 9,loss=0.036644503474235535
After epoch 10,loss=0.029920388013124466
After epoch 11,loss=0.023427007719874382
After epoch 12,loss=0.01968187279999256
After epoch 13,loss=0.016636686399579048
After epoch 14,loss=0.01425135973840952
After epoch 15,loss=0.012160095386207104
After epoch 16,loss=0.010611655190587044
After epoch 17,loss=0.009309808723628521
After epoch 18,loss=0.0081344498321414
After epoch 19,loss=0.007255277596414089
After epoch 20,loss=0.006278789136558771


**As we can see above, our tarin loss almost comes down to zero. This shows the learning process is not an issue. Time to collect more trainig data.**